### Run this if you are using Google Colab

In [ ]:
# !pip install langchain langchain-openai langchain_mcp_adapters fastmcp

In [ ]:
# import os
# from google.colab import userdata

# os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
# os.environ["LANGSMITH_API_KEY"] = userdata.get("LANGSMITH_API_KEY")
# os.environ["LANGSMITH_PROJECT"] = "ncit-workshop"
# os.environ["LANGSMITH_TRACING"] = "true"
# os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
# os.environ["QDRANT_API_KEY"] = userdata.get("QDRANT_API_KEY")
# os.environ["QDRANT_URL"] = "qdrant-host"

### Run this if you are running VSCode

In [ ]:
import sys
from pathlib import Path

sys.path.append(str(Path().resolve().parent))
from core import load_vault_env

load_vault_env()

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
from langchain.agents import create_agent
from langchain_mcp_adapters.client import MultiServerMCPClient

In [ ]:
server_config = {
    "campus_ops": {
        "command": "python",
        "args": ["server.py"],
        "transport": "stdio",
    },
    "filesystem": {
        "command": "npx",
        "args": ["-y", "@modelcontextprotocol/server-filesystem", "."],
        "transport": "stdio",
    },
}

client = MultiServerMCPClient(server_config)

In [ ]:
tools = await client.get_tools()

for tool in tools:
    print(f"[{tool.name}]\n {tool.description}\n")

#### Create Agent

In [ ]:
agent = create_agent(
    model="gpt-4.1-mini",
    tools=tools,
    system_prompt="You are the Campus Defense AI. Manage threats and personnel.",
)

In [ ]:
async def run_agent(query: str):
    async for step in agent.astream(
        {"messages": [{"role": "user", "content": query}]},
        stream_mode="values",
    ):
        step["messages"][-1].pretty_print()

#### Scan & Deploy

In [ ]:
# Mission - Threat Assessment
# Task: Find threats and deploy the right person.
query = (
    "Scan the campus for active threats. "
    "If you find a threat in the CS Lab, deploy 'Alice' (our CS major) to fix it."
)

await run_agent(query)

#### Recruitment

In [ ]:
query = (
    "We need more muscle. Recruit a new hunter named 'Thor'. "
    "He is an 'Engineering' major and uses a 'Mjolnir Mallet'. "
    "After recruiting him, list all available hunters to confirm he is there."
)
await run_agent(query)

#### Cross Server Operation

Demonstrating using the Database Server AND the FileSystem Server in one go.

In [ ]:
# Mission - Archiving Data
# Task: Read from DB, Write to File.
query = (
    "Create a summary report of all ACTIVE threats remaining in the database. "
    "Save this report to a text file named 'threat_report.txt' in the current directory."
)
await run_agent(query)